## Euler Problem 115
## Counting block combinations II

A row measuring n units in length has red blocks with a minimum length of m units placed on it, such that any two red blocks (which are allowed to be different lengths) are separated by at least one black square.

Let the fill-count function, F(m, n), represent the number of ways that a row can be filled.

For example, F(3, 29) = 673135 and F(3, 30) = 1089155.

That is, for m = 3, it can be seen that n = 30 is the smallest value for which the fill-count function first exceeds one million.

In the same way, for m = 10, it can be verified that F(10, 56) = 880711 and F(10, 57) = 1148904, so n = 57 is the least value for which the fill-count function first exceeds one million.

For m = 50, find the least value of n for which the fill-count function first exceeds one million.

In [1]:
using Primes, Lazy, Combinatorics, BigCombinatorics

This looks to be a partition problem. We will want to partition the blocks so that all blocks of size > 2 are seperated from each other by a '1'.

In [2]:
@>> collect(partitions(7))

15-element Array{Any,1}:
 [7]                  
 [6, 1]               
 [5, 2]               
 [5, 1, 1]            
 [4, 3]               
 [4, 2, 1]            
 [4, 1, 1, 1]         
 [3, 3, 1]            
 [3, 2, 2]            
 [3, 2, 1, 1]         
 [3, 1, 1, 1, 1]      
 [2, 2, 2, 1]         
 [2, 2, 1, 1, 1]      
 [2, 1, 1, 1, 1, 1]   
 [1, 1, 1, 1, 1, 1, 1]

We need to make sure that we filter out '2's since these represent blocks of size 2. Moreover, we need to make sure that there are sufficient '1' to partition the other numbers.

In [3]:
function enoughones(ns,m)
  return all(map(n-> !(n in 2:m-1),ns)) && (count(isone,ns) >= div(length(ns),2))
end

enoughones (generic function with 1 method)

In [4]:
@>> collect(partitions(7)) filter(a-> enoughones(a,3))

7-element Array{Any,1}:
 [7]                  
 [6, 1]               
 [5, 1, 1]            
 [4, 1, 1, 1]         
 [3, 3, 1]            
 [3, 1, 1, 1, 1]      
 [1, 1, 1, 1, 1, 1, 1]

Need to filter out rows containing 2's and rows without enough ones 

Logic for Combinations: Using the bars and stars approach. Where '1' are stars and other digits 'd' are bars. So for example with [3,3,1,1,1] there are 3 ones, with 2 gaps between and an additional spot on either end, i.e., 5 where we can place the 2 other digits. Generally, there are (n_ones + 1) spots to place the 'ds':

`C(ones + 1, nd)`

Next the combinations of digits that can fit into the (ones+1) needs to be taken into account:

`nds!/(nd1!*nd2!*..)`

Finally, 

`C(ones + 1, nd) * nds!/(nd1!*nd2!*..)`

In [5]:
function combs(ds)
    n_ds = length(ds)
    n_ds == 1 && return(1)
    fs = frequencies(ds)
    n_ones = fs[1]
    n_rest = n_ds - n_ones
    parts  = div(factorial(n_rest), prod([factorial(n) for (d,n) in pairs(fs) if d != 1 ]))
    choose = binomial(n_ones+1, n_rest)
    return(parts * choose)                    
end

combs (generic function with 1 method)

In [6]:
function F(m,n)
    @>> collect(partitions(n)) filter(a-> enoughones(a,m)) map(combs) sum
end

F (generic function with 1 method)

In [7]:
@time F(3,50)

  0.385823 seconds (1.85 M allocations: 123.131 MiB, 8.53% gc time)


16475640049

In [8]:
@time F(3,29)

  0.001510 seconds (30.55 k allocations: 2.153 MiB)


673135

In [9]:
@time F(3,30)

  0.001994 seconds (37.27 k allocations: 2.630 MiB)


1089155

In [10]:
@time F(10,56)

  0.416257 seconds (2.64 M allocations: 194.339 MiB, 58.85% gc time)


880711

In [11]:
@time F(10,100)

4129.922871 seconds (953.77 M allocations: 79.765 GiB, 96.96% gc time)


102830344689

In [12]:
@time F(50,100)

8092.704161 seconds (952.85 M allocations: 79.690 GiB, 97.94% gc time)


1327

In [ ]:
@time F(50,110)

In [ ]:
@time F(50,120)

In [ ]:
partitions(10)

In [ ]:
#@>> Lazy.range(3) map(m-> F(50,m)) take(50)

### Utilities

`Printing Utilities`

In [ ]:
macro print_seq(aseq) 
    :(for p in $aseq println(p) end) 
end

In [ ]:
macro print_seq(aseq, lines)
    :(for p in take($aseq,$lines) println(p) end)
end

`stats(dict) returns attributes of dict`

In [ ]:
function stats(dict)
    e = last(sort(collect(keys(dict))))
    #m = last(sort(collect(values(dict))))
    return("Dict[$e] = $(dict[e]), Size = $(length(dict))")
end

`sorted_dict(dict) returns a sorted array of dictionary pairs`

In [ ]:
sorted_dict(dict) = [(k,dict[k]) for k in sort(collect(keys(dict)))]